In [ ]:
import os
import sys
import numpy as np


# Root directory of the project
ROOT_DIR = os.path.abspath("../../")
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from openimages import OpenImageDataset
from openimages import OpenImagesConfig
import openimages as oim
from tqdm import tqdm

In [ ]:
class InferenceConfig(OpenImagesConfig):
    """
    """
    # Give the configuration a recognizable name
    NAME = "openimages"

    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    
inference_config = InferenceConfig()

In [ ]:
# Recreate the model in inference mode
DEVICE = "/cpu:0"  # /cpu:0 or /gpu:0

import tensorflow as tf

with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", 
                              config=inference_config,
                              model_dir=MODEL_DIR)
  
# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
#model_path = model.find_last()
model_path = os.path.join(ROOT_DIR, 'logs/openimages20190926T0931/mask_rcnn_openimages_0018.h5')

# Load trained weights
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

In [ ]:
PATH = os.path.abspath(os.path.join(ROOT_DIR, '../data/'))

# Validation dataset
dataset = OpenImageDataset()
dataset.load_dataset(PATH, 'kaggle-test')
dataset.prepare()

In [ ]:
all_rows = ["ImageID,ImageWidth,ImageHeight,PredictionString"]

for image_id in tqdm(dataset.image_ids):

    original_image = dataset.load_image(image_id)
    
    results = model.detect([original_image], verbose=0)

    r = results[0]
    n_detections = r['masks'].shape[2]
    
    detections = []
    source_image = dataset.image_info[image_id]['id']
    image_size = dataset.image_info[image_id]['size']
    
    for detection_index in range(n_detections):

        mask = r['masks'][:, :, detection_index]
        score = r['scores'][detection_index]
        class_id = r['class_ids'][detection_index]
        bbox = r['rois'][detection_index, :]

        label = dataset.get_source_class_id(class_id, 'openimages')

        
        #true_mask = oim.unmold_mask(mask, bbox, image_size)
        true_mask = mask
        
        rle_mask = oim.encode_binary_mask(true_mask)
       
        detections.extend([label, str(score), rle_mask.decode('ascii')])
        
    detections_str = " ".join(detections)
    row = ",".join([source_image, str(image_size[0]), str(image_size[1]), detections_str])
    all_rows.append(row)


In [ ]:
# complete with empty detections if necessary

In [ ]:
if len(all_rows) < len(dataset.image_ids) + 1:
    for image_info in dataset.image_info[len(all_rows)-1:]:
        all_rows.append(",".join([image_info['id'], '-1', '-1', '']))

In [ ]:
with open("submission.csv", "w") as fid:
    fid.write("\n".join(all_rows))